### Using `schypo` to analyze Interferon-B response in monocytes

To install `schypo` in the pre-release version (for Ye Lab members), install it directly from github

In [21]:
import scanpy as sc
from schypo import schypo, util

In [3]:
fig_path = '/data/home/Github/scrna-parameter-estimation/figures/fig4/'
data_path = '/data/parameter_estimation/'

### Read IFN data and filter for monocytes

For `schypo`, we need the raw count matrix. Preferrably, feed the one with all genes so that we can choose what genes to look at. 

One of the columns in `adata.obs` should be the discrete groups to compare mean, variability, and co-variability across. In this case, it's called `stim`. 

The column containing the covariate that you want p-values for should either:
- Be binary (aka the column only contains two unique values, such as 'A' and 'B'. Here, the values are either 'stim' or 'ctrl'.
- Be numeric (aka the column contains -1, 0, -1 for each genotype value). 

In [14]:
adata = sc.read(data_path + 'interferon_filtered.h5ad')
adata = adata[adata.obs.cell == 'CD14+ Monocytes'].copy()
print(adata)

AnnData object with n_obs × n_vars = 5341 × 35635 
    obs: 'tsne1', 'tsne2', 'ind', 'stim', 'cluster', 'cell', 'multiplets', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'cell_type'
    var: 'gene_ids', 'mt', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'cell_type_colors'
    obsm: 'X_tsne'


In [48]:
adata.obs[['ind', 'stim', 'cell']].sample(5)

,ind,stim,cell
index,,,
CCTTTAGATGGAAA-1,1039,ctrl,CD14+ Monocytes
CGCCATTGCTGGTA-1,1488,stim,CD14+ Monocytes
AAACCGTGCGCCTT-1,1015,ctrl,CD14+ Monocytes
TTGGAGTGTATTCC-1,1244,ctrl,CD14+ Monocytes
GTGTACGACATGAC-1,107,ctrl,CD14+ Monocytes


### Create groups for hypothesis testing and compute 1D parameters

`schypo` creates groups of cells based on anything that should be considered a reasonable group; here, we just divide the cells into `stim` and `ctrl`. But we can easily further divide the cells into individuals by adding the `ind` column to the `label_columns` argument when calling `create_groups`.

`q` is the rough estimate of the overall UMI efficiency across both sampling and sequencing. If `s` is the sequencing saturation, multiply `s` by 0.07 for 10X v1, 0.15 for v2, and 0.25 for v3. 

By default, `schypo` will consider all genes whose expression is high enough to calculate an accurate variance. If you wish to include less genes, increase `filter_mean_thresh`.

In [15]:
schypo.create_groups(adata, label_columns=['stim'], inplace=True, q=0.07)

In [16]:
schypo.compute_1d_moments(
    adata, 
    inplace=True, 
    filter_genes=True, # Filter the AnnData object
    filter_mean_thresh=0.05, # minimum raw mean of each gene within a group for the gene to be considered 
    min_perc_group=.9) # percentage of groups that satisfy the condition for a gene to be considered. 

### Perform 1D hypothesis testing

`formula_like` determines the linear model that is used for hypothesis testing, while `cov_column` is used to pick out the variable that you actually want p-values for. 

`num_cpus` controls how many CPUs to parallelize this operation for. In general, I recommend using 3-6 CPUs for reasonable peformance on any of the AWS machines that we have access to (I'm currently using a c5.2xlarge instance (8 vCPUs). 

In [18]:
schypo.ht_1d_moments(
    adata, 
    formula_like='1 + stim',
    cov_column='stim', 
    num_boot=5000, 
    verbose=1,
    num_cpus=6)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   10.2s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   20.9s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:   36.5s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:   57.2s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 2337 out of 2337 | elapsed:  1.8min finished


In [22]:
result_1d = schypo.get_1d_ht_result(adata)
result_1d['de_fdr'] = util._fdrcorrect(result_1d['de_pval'])
result_1d['dv_fdr'] = util._fdrcorrect(result_1d['dv_pval'])

In [26]:
result_1d.query('de_fdr < 0.1').sort_values('de_fdr').head(10)

,gene,de_coef,de_pval,dv_coef,dv_pval,de_fdr,dv_fdr
958,LAMTOR4,-0.664377,5.416919e-22,0.162332,0.664267,1.265934e-18,0.871641
444,BRK1,-1.091400,3.158007e-14,-0.544764,0.175565,3.690131e-11,0.568951
2196,PLAUR,-1.104445,6.723418e-13,-0.052713,0.807838,5.237542e-10,0.917356
204,CD48,0.718466,3.825761e-12,-0.757255,0.000006,2.235201e-09,0.001010
1384,COX8A,-0.635169,7.446282e-11,0.452424,0.269546,3.480392e-08,0.647409
1403,DRAP1,0.801873,8.985642e-11,-0.292609,0.316737,3.499908e-08,0.697000
696,UBE2B,0.232309,2.620131e-10,-0.916129,0.353129,8.747494e-08,0.720164
236,RNPEP,-0.765233,6.329750e-10,0.969703,0.027594,1.643625e-07,0.247082
267,RPS7,-0.994337,6.168798e-10,0.300835,0.149570,1.643625e-07,0.536936
1678,COX16,-0.522020,9.780651e-10,0.293606,0.626675,2.285738e-07,0.858463


In [28]:
result_1d.query('dv_fdr < 0.1').sort_values('dv_fdr').head(10)


,gene,de_coef,de_pval,dv_coef,dv_pval,de_fdr,dv_fdr
1436,CARD16,0.473250,2.476202e-06,-0.917231,1.803329e-09,0.000009,0.000004
1041,RPS4X,-0.645258,1.046123e-04,0.478713,1.866258e-07,0.000165,0.000218
1100,IDO1,3.859922,1.835622e-08,-1.506381,3.362157e-07,0.000001,0.000262
1905,CCL2,1.450735,3.386626e-07,-1.074444,5.098625e-07,0.000003,0.000298
0,ISG15,4.524188,3.335918e-07,-2.205038,1.429808e-06,0.000003,0.000638
179,S100A4,-0.625340,4.925540e-05,-0.546267,1.638266e-06,0.000081,0.000638
1317,IFITM3,3.292916,3.296029e-08,-2.206570,2.058045e-06,0.000001,0.000687
1160,RPS6,-0.825058,7.754184e-08,0.549673,3.158067e-06,0.000002,0.000826
591,IL8,-2.041506,4.786861e-07,0.865848,3.182921e-06,0.000004,0.000826
723,RPS14,-0.399055,6.884207e-07,0.419926,3.840718e-06,0.000005,0.000898


### Perform 2D hypothesis testing

For differential co-variability testing, we can specify which genes you want to perform HT on. It takes two lists of genes, `gene_1` and `gene_2`, and it will calculate correlations between every gene in `gene_1` and every gene in `gene_2`. Here, we focus on 3 transcription factors and their correlations to rest of the transcriptome. 

Similar to the 1D case, 2D hypothesis testing scales with the number of pairs of genes to test. If you have a smaller set of candidate genes, it will run faster.

In [38]:
schypo.compute_2d_moments(
    adata, 
    ['JUN', 'FOS', 'STAT1'], 
    adata.var.index.tolist())

In [39]:
schypo.ht_2d_moments(
    adata, 
    formula_like='1 + stim', 
    cov_column='stim', 
    num_cpus=6, 
    num_boot=5000)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   10.2s
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed:   22.3s
[Parallel(n_jobs=6)]: Done 500 tasks      | elapsed:   37.8s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:   58.7s
[Parallel(n_jobs=6)]: Done 1140 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 1556 tasks      | elapsed:  1.9min
[Parallel(n_jobs=6)]: Done 2036 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done 2580 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  3.9min
[Parallel(n_jobs=6)]: Done 3860 tasks      | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done 4596 tasks      | elapsed:  5.5min
[Parallel(n_jobs=6)]: Done 5396 tasks      | elapsed:  6.5min
[Parallel(n_jobs=6)]: Done 6260 tasks      | elapsed:  7.7min
[Parallel(n_jobs=6)]: Done 7005 out of 7005 | elapsed:  8.7min

In [41]:
result_2d = schypo.get_2d_ht_result(adata)

In [43]:
result_2d.sort_values('corr_fdr').head(10)

,gene_1,gene_2,corr_coef,corr_pval,corr_fdr
5762,STAT1,DOK2,-0.408536,1.124117e-06,0.003939
3023,FOS,HINT1,-0.322596,6.272982e-07,0.003939
62,JUN,PTP4A2,-0.298890,2.131984e-06,0.004980
3862,FOS,GLIPR1,-0.287573,4.682828e-06,0.006978
3536,FOS,FKBP15,-0.320691,4.978389e-06,0.006978
1437,JUN,RDX,-0.292184,1.518471e-05,0.016838
4052,FOS,KLF13,-0.384214,1.887668e-05,0.016838
1412,JUN,TCIRG1,0.437781,1.922116e-05,0.016838
6369,STAT1,LGMN,0.264664,2.542207e-05,0.019795
3551,FOS,ENG,-0.350422,2.383069e-04,0.167005
